# Normalize Cell Health Labels

**Gregory Way, 2019**

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline

In [3]:
# Function to scale cell health target variables
def mad_scale(x):
    x_median = np.nanmedian(x)
    x_mad = x.mad()
    x_mad_scale = (x - x_median) / x_mad
    return x_mad_scale

In [4]:
file = os.path.join("data", "labels", "cell_health_labels.tsv")
label_df = pd.read_csv(file, sep='\t')

print(label_df.shape)
label_df.head(2)

(2302, 75)


,cell_id,guide,plate_name,well_col,well_row,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,ES2,AKT1-1,Plate 1,3,A,0.04287,0.007976,0.003988,1003,1.777,...,1465.0,0.02710,0.01190,0.01520,1.640,0.03173,0.01652,0.9683,NaN,NaN
1,ES2,AKT1-1,Plate 1,22,O,0.02635,0.005988,0.005988,835,1.582,...,1575.0,0.03169,0.01463,0.01706,1.405,0.03961,0.02255,0.9598,279.6,2083.0


In [5]:
# Apply normalization by plate
normalized_label_df = (
    label_df
    .drop(["guide", "well_col", "well_row"], axis="columns")
    .groupby(["cell_id", "plate_name"])
    .transform(mad_scale)
)

normalized_label_df = pd.concat(
    [
        label_df.loc[:, ["cell_id", "guide", "plate_name", "well_col", "well_row"]],
        normalized_label_df
    ],
    axis="columns"
)

print(normalized_label_df.shape)
normalized_label_df.head(2)

(2302, 75)


,cell_id,guide,plate_name,well_col,well_row,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,ES2,AKT1-1,Plate 1,3,A,1.068472,-0.941982,-1.382715,-0.648839,0.469332,...,0.366524,-0.529604,-0.739827,-0.385889,-1.260855,-0.287531,-0.029597,0.288934,NaN,NaN
1,ES2,AKT1-1,Plate 1,22,O,-0.409849,-1.359614,-0.866584,-1.035381,-0.674665,...,0.708198,-0.420547,-0.536226,-0.324803,-1.466192,-0.096168,0.480319,0.082553,-0.744996,0.023459


In [6]:
# Write to file
file = os.path.join("data", "labels", "normalized_cell_health_labels.tsv")
normalized_label_df.to_csv(file, index=False, sep='\t')